In [23]:
import pandas as pd
import numpy as np
import simulation_func as sf  # self-written simulation functions
import time

<module 'simulation_func' from '/Users/tsung_macbook/Dropbox/Ad_python/final/simulation_func.py'>

## simulation_setup

* This script sets up the environment required for our simulation
* The output will be a Python dictionary saved at "simulated_values.pkl"
```
simulated_values.keys() == dict_keys(['simulated_time', 'simulated_distance', 'simulated_trip', 'wait_time', 'simulated_fare'])
```
**Time simulation**

- turn buffer to numpy
- avoid repetitive filtering
- try to use numba.jit but not compatible with interp1d function
- multiprocessing


**Trip simulation**

- multiprocessing

**wait time**
- turn to np.array

**Distance simulation**

- pandas group by 
- calcualte all at once by pandas apply

**Fare simulation**
- numpy broadcasting

In [158]:
# !python simulation_setup.py   # simulation setup

Time took for time simulation: 11.989735126495361
Time took for trip simulation: 29.841912984848022
The R squared of the model is:  0.8705780665239197
Time took for distance simulation: 0.23627686500549316
Time took for distance & wait time simulation: 0.0191953182220459
Finish simulation setup, total time spent: 43.08929991722107


In [6]:
simulated_values = sf.load_py("simulated_values.pkl") # load simulation setup values

In [77]:
start_time = time.time()
seed = 0                           # set specific seed for reproduction
sim1_zone_freq = np.zeros(40)      # buffer for saving 40 zones frequency
iter_per_zone = 100                # no. of iteration for 1 starting zone
sim1_trips = np.zeros((40*iter_per_zone, 3)) # empty 2D array to store results
for i_start in range(40):
    for j in range(iter_per_zone):
        seed += 1 
        sim1_result = sf.driver_sim1(simulated_values, i_start, verbose=0, seed=seed)
        sim1_zone_freq += sim1_result["zone_freq"]
        sim1_trips[i_start*iter_per_zone + j, :] = [i_start, sim1_result["num_of_trip"], sim1_result["total_fare"]]
        # save i_start, sim1_result["total_fare"], sim1_result["num_of_trip"] into a array
print(f"Time spent for {iter_per_zone} trips per zone: {time.time()-start_time}")
sim1_df = pd.DataFrame(sim1_trips, columns=["start_zone", "num_of_trip", "total_fare"])
sim1_df.start_zone = sim1_df.start_zone.astype("i")
sim1_df.head()


Time spent for 100 trips per zone: 1.2216053009033203


,start_zone,num_of_trip,total_fare
0,0,53.0,449.996712
1,0,71.0,605.584812
2,0,60.0,525.350425
3,0,61.0,501.770993
4,0,58.0,453.616456


In [79]:
# some simple summary:
sim1_df.groupby('start_zone', as_index=True).describe()

num_of_trip                                                   \
                 count   mean       std   min    25%   50%    75%   max   
start_zone                                                                
0                100.0  60.71  4.682430  50.0  58.00  60.0  64.00  74.0   
1                100.0  60.67  4.968700  48.0  57.00  60.0  64.00  80.0   
2                100.0  61.17  4.413146  51.0  59.00  61.0  64.00  73.0   
3                100.0  60.36  4.768415  47.0  57.00  60.0  64.00  72.0   
4                100.0  61.23  4.730847  50.0  58.00  61.0  64.00  72.0   
5                100.0  60.39  4.496677  49.0  58.00  60.5  63.00  71.0   
6                100.0  61.10  5.170819  49.0  58.00  60.5  65.00  73.0   
7                100.0  60.48  5.256031  48.0  57.00  61.0  64.00  74.0   
8                100.0  60.81  4.441983  52.0  58.00  60.5  63.25  73.0   
9                100.0  61.02  4.249730  50.0  59.00  61.0  63.00  74.0   
10               100.0  61.80  4.629724  54.0  58.75  62.0  65.00  73.0   
11               100.0  61.20  4.946175  50.0  58.00  61.0  64.00  75.0   
12               100.0  60.88  4.961020  48.0  57.75  61.0  63.25  71.0   
13               100.0  61.99  4.700086  50.0  59.00  62.0  65.00  72.0   
14               100.0  61.17  4.192478  51.0  58.00  61.5  64.00  71.0   
15               100.0  60.29  5.545578  44.0  57.00  60.0  63.25  75.0   
16               100.0  60.57  5.007580  43.0  57.00  61.0  64.00  73.0   
17               100.0  60.32  4.543571  47.0  57.00  60.0  63.00  72.0   
18               100.0  61.87  4.498384  52.0  59.00  62.0  65.25  73.0   
19               100.0  60.29  5.737164  46.0  56.00  60.0  65.00  72.0   
20               100.0  61.46  4.717087  50.0  59.00  61.0  64.00  72.0   
21               100.0  60.81  4.538711  50.0  58.00  61.0  64.00  74.0   
22               100.0  60.86  4.966595  48.0  57.75  61.0  64.25  72.0   
23               100.0  61.41  4.470091  52.0  58.00  61.5  64.25  72.0   
24               100.0  60.38  4.842530  49.0  56.00  60.0  64.00  70.0   
25               100.0  60.49  4.985828  48.0  57.00  60.0  63.25  74.0   
26               100.0  61.65  4.932627  47.0  58.00  61.5  66.00  73.0   
27               100.0  60.90  4.783178  50.0  58.00  60.5  64.00  80.0   
28               100.0  60.90  5.294365  45.0  57.00  60.0  64.00  74.0   
29               100.0  60.90  4.305270  48.0  58.75  61.0  64.00  75.0   
30               100.0  60.70  4.255715  49.0  58.00  61.0  63.00  74.0   
31               100.0  60.29  4.920161  51.0  56.00  60.0  64.00  70.0   
32               100.0  61.39  4.982586  42.0  58.75  61.5  64.00  74.0   
33               100.0  60.26  4.663679  49.0  57.00  60.0  63.00  76.0   
34               100.0  60.88  4.447857  53.0  58.00  61.0  63.00  74.0   
35               100.0  60.69  4.804659  52.0  57.00  61.0  64.00  75.0   
36               100.0  60.59  5.239892  43.0  58.00  61.0  64.00  70.0   
37               100.0  61.78  4.232104  50.0  58.75  62.0  65.00  72.0   
38               100.0  60.49  4.802135  50.0  57.00  60.0  64.00  71.0   
39               100.0  61.03  4.676300  47.0  58.00  61.0  64.00  75.0   

           total_fare                                                 \
                count        mean        std         min         25%   
start_zone                                                             
0               100.0  494.083472  47.285553  415.990478  461.515492   
1               100.0  495.948827  47.960193  383.974648  464.210243   
2               100.0  501.506374  44.863997  398.347353  474.271489   
3               100.0  488.787921  48.551976  397.849605  454.732621   
4               100.0  479.158653  43.840605  379.448130  453.318875   
5               100.0  488.624596  41.554060  390.395627  465.465616   
6               100.0  496.231246  48.734611  358.570273  464.868544   
7               100.0  498.028455  48.083314  389.713160  466.623

In [99]:
# TODO: add folium plot
df = pd.DataFrame(np.vstack((np.arange(40), sim1_zone_freq)).T, columns=["zone", "frequency"])
df.head()

,zone,frequency
0,0.0,3923.0
1,1.0,2634.0
2,2.0,4643.0
3,3.0,3786.0
4,4.0,1519.0
